# Faster TensorFlow Text Generation with XLA

This notebook is a companion to the [Hugging Face blog post with the same title](https://huggingface.co/blog/tf-xla-generate). It is meant to illustrate how to use XLA with TensorFlow text generation.

This notebook has the following code cells:
1. Prepating the environment
2. A stand-alone TF XLA generate example

⚠️ If you are running this on colab, you might not have access to a GPU. The benefits of XLA are best observed with a GPU!

In [ ]:
# Preparing the environment
!pip install transformers>=4.21.0

In [ ]:
# A stand-alone TF XLA generate example -- if you have a machine with a good GPU, I highly recommend you to try this example there!
import time
import tensorflow as tf
from transformers import AutoConfig, AutoTokenizer, TFAutoModelForCausalLM, TFAutoModelForSeq2SeqLM

# 1. Load model and tokenizer
model_name = "t5-small"
config = AutoConfig.from_pretrained(model_name)

if config.is_encoder_decoder:
    tokenizer_init_kwargs = {}
    model_class_for_generation = TFAutoModelForSeq2SeqLM
else:  # remember: decoder-only models need left-padding
    tokenizer_init_kwargs = {"padding_side": "left", "pad_token": "</s>"}
    model_class_for_generation = TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, **tokenizer_init_kwargs)
model = model_class_for_generation.from_pretrained(model_name)

# 2. Prepare tokenization and generation arguments -- don't forget padding to avoid retracing!
tokenization_kwargs = {"pad_to_multiple_of": 32, "padding": True, "return_tensors": "tf"}
generation_kwargs = {"num_beams": 4, "max_new_tokens": 32}

# 3. Create your XLA generate function a̶n̶d̶ ̶m̶a̶k̶e̶ ̶P̶y̶T̶o̶r̶c̶h̶ ̶e̶a̶t̶ ̶d̶u̶s̶t̶
# This is the only change with respect to original generate workflow!
xla_generate = tf.function(model.generate, jit_compile=True)

# 4. Generate! Remember -- the first call will be slow, but all subsequent calls will be fast if you've done things right.
input_prompts = [
    f"translate English to {language}: I have four cats and three dogs." for language in ["German", "French", "Romanian"]
]
for input_prompt in input_prompts:
    tokenized_text = tokenizer([input_prompt], **tokenization_kwargs)
    start = time.time_ns()
    generated_text = xla_generate(**tokenized_text, **generation_kwargs)
    end = time.time_ns()
    decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    print(f"Original prompt -- {input_prompt}")
    print(f"Generated -- {decoded_text}")
    print(f"Execution time -- {(end - start) / 1e6:.1f} ms\n")